In [9]:
import os
import pandas as pd
from sqlalchemy import create_engine
import urllib

# --- ⚙️ SETTINGS ---
# Folder containing your Excel files
excel_folder = r"D:\Data\Markting_olist"   # e.g. folder with multiple .xlsx files

# بيانات الاتصال
server = "almagraby.database.windows.net"
database = "Olist_dataset"
username = "DEPI_Projecte"
password = "Almaghraby@240https://api.cognitive.microsofttranslator.com"
driver = "ODBC Driver 18 for SQL Server"

# ترميز كلمة المرور لتفادي مشاكل الرموز الخاصة مثل @
encoded_password = urllib.parse.quote_plus(password)

# بناء الاتصال بشكل آمن
conn_str = (
    f"mssql+pyodbc://{username}:{encoded_password}@{server}:1433/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=no"
)

# اختبار الاتصال
engine = create_engine(conn_str)

try:
    with engine.connect() as conn:
        print("✅ تم الاتصال بـ Azure SQL بنجاح!")
except Exception as e:
    print("❌ فشل الاتصال:", e)

# --- 📂 Loop through all Excel files in the folder ---
for file_name in os.listdir(excel_folder):
    if file_name.endswith(".xlsx"):
        file_path = os.path.join(excel_folder, file_name)
        print(f"\n📘 Processing file: {file_name}")

        # Read all sheets in workbook
        sheets = pd.read_excel(file_path, sheet_name=None)

        # Upload each sheet as a table
        for sheet_name, df in sheets.items():
            # Clean table name: file name + sheet name
            base_name = os.path.splitext(file_name)[0].replace(" ", "_")
            clean_sheet = sheet_name.replace(" ", "_")
            table_name = f"{base_name}_{clean_sheet}"

            print(f"   ⬆️ Uploading sheet '{sheet_name}' → table '{table_name}' ...")
            try:
                df.to_sql(table_name, con=engine, if_exists="replace", index=False)
                print(f"   ✅ Uploaded successfully!")
            except Exception as e:
                print(f"   ❌ Failed to upload '{sheet_name}' from '{file_name}': {e}")

print("\n🎉 All Excel files processed successfully!")


✅ تم الاتصال بـ Azure SQL بنجاح!

📘 Processing file: olist_closed_deals_dataset.xlsx
   ⬆️ Uploading sheet 'olist_closed_deals_dataset' → table 'olist_closed_deals_dataset_olist_closed_deals_dataset' ...
   ✅ Uploaded successfully!

📘 Processing file: olist_marketing_qualified_leads_dataset.xlsx
   ⬆️ Uploading sheet 'olist_marketing_qualified_leads' → table 'olist_marketing_qualified_leads_dataset_olist_marketing_qualified_leads' ...
   ✅ Uploaded successfully!

🎉 All Excel files processed successfully!


In [7]:
import urllib
from sqlalchemy import create_engine

# بيانات الاتصال
server = "almagraby.database.windows.net"
database = "Olist_dataset"
username = "DEPI_Projecte"
password = "Almaghraby@240"
driver = "ODBC Driver 18 for SQL Server"

# ترميز كلمة المرور لتفادي مشاكل الرموز الخاصة مثل @
encoded_password = urllib.parse.quote_plus(password)

# بناء الاتصال بشكل آمن
conn_str = (
    f"mssql+pyodbc://{username}:{encoded_password}@{server}:1433/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=no"
)

# اختبار الاتصال
engine = create_engine(conn_str)

try:
    with engine.connect() as conn:
        print("✅ تم الاتصال بـ Azure SQL بنجاح!")
except Exception as e:
    print("❌ فشل الاتصال:", e)


✅ تم الاتصال بـ Azure SQL بنجاح!


In [10]:
from sqlalchemy import create_engine
import urllib

In [11]:
import pandas as pd
import pymssql
import requests
import uuid
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Azure SQL Server Configuration
SERVER = 'almagraby.database.windows.net'
DATABASE = 'Olist_dataset'
USERNAME = 'DEPI_Projecte'
PASSWORD = 'Almaghraby@240'
TABLE_NAME = 'olist_geolocation_dataset_olist_geolocation_dataset'  # Table containing geolocation_city column

# Azure Translator Configuration
AZURE_TRANSLATOR_KEY = 'AunHlUi5xLkxOBfcQnwIYmiLxh2PztKTD5b5Kbfu4uiLHt1alQKdJQQJ99BJACF24PCXJ3w3AAAbACOGZEWL'  # From Azure Portal
AZURE_TRANSLATOR_REGION = 'uaenorth'  # Your translator region (e.g., eastus, westeurope)
AZURE_TRANSLATOR_ENDPOINT = 'https://almagraby.cognitiveservices.azure.com/'

# Performance Settings
BATCH_SIZE = 100
MAX_WORKERS = 5
DB_UPDATE_BATCH_SIZE = 50  # CRITICAL: Smaller batches for UPDATE statements

def get_connection():
    """Create a new database connection with retry logic"""
    max_retries = 3
    retry_delay = 5
    
    for attempt in range(max_retries):
        try:
            conn = pymssql.connect(
                server=SERVER,
                user=USERNAME,
                password=PASSWORD,
                database=DATABASE,
                port=1433,
                tds_version='7.4',
                timeout=60,
                login_timeout=60
            )
            return conn
        except pymssql.OperationalError as e:
            error_code = e.args[0] if e.args else None
            
            if error_code == 40615:
                # Firewall blocking
                print(f"\n⚠️  FIREWALL ERROR (Attempt {attempt + 1}/{max_retries})")
                print("=" * 70)
                print("Your Azure SQL Server firewall is blocking this connection.")
                print("\nTO FIX THIS:")
                print("1. Get your current IP:")
                
                try:
                    import requests
                    current_ip = requests.get('https://api.ipify.org', timeout=5).text
                    print(f"   Your IP: {current_ip}")
                except:
                    print("   Run: import requests; print(requests.get('https://api.ipify.org').text)")
                
                print("\n2. Add to Azure Firewall:")
                print("   • Azure Portal → SQL Server → Networking")
                print("   • Click '+ Add firewall rule'")
                print(f"   • Start IP: {current_ip if 'current_ip' in locals() else '<your_ip>'}")
                print(f"   • End IP: {current_ip if 'current_ip' in locals() else '<your_ip>'}")
                print("   • Click 'Save' and wait 2-5 minutes")
                print("\n3. OR Enable 'Allow Azure services' (Recommended):")
                print("   • Azure Portal → SQL Server → Networking")
                print("   • Toggle ON: 'Allow Azure services and resources to access this server'")
                print("   • Click 'Save'")
                print("=" * 70)
                
                if attempt < max_retries - 1:
                    print(f"\nWaiting {retry_delay} seconds before retry...")
                    time.sleep(retry_delay)
                else:
                    print("\n❌ Max retries reached. Please fix firewall and re-run.")
                    raise
            else:
                # Other connection errors
                print(f"\n❌ Connection Error: {e}")
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    raise
        except Exception as e:
            print(f"\n❌ Unexpected Error: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                raise
    
    raise Exception("Failed to connect after all retries")

def translate_batch_azure(cities):
    """Translate a batch of cities using Azure Translator API"""
    if not cities:
        return {}
    
    valid_cities = [city for city in cities if pd.notna(city) and city != '']
    
    if not valid_cities:
        return {city: city for city in cities}
    
    path = '/translate'
    constructed_url = AZURE_TRANSLATOR_ENDPOINT + path
    
    params = {
        'api-version': '3.0',
        'from': 'auto',
        'to': 'en'
    }
    
    headers = {
        'Ocp-Apim-Subscription-Key': AZURE_TRANSLATOR_KEY,
        'Ocp-Apim-Subscription-Region': AZURE_TRANSLATOR_REGION,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    
    body = [{'text': str(city)} for city in valid_cities]
    translations = {}
    
    try:
        response = requests.post(constructed_url, params=params, headers=headers, json=body)
        response.raise_for_status()
        results = response.json()
        
        for city, result in zip(valid_cities, results):
            translated_text = result['translations'][0]['text']
            translations[city] = translated_text
        
        for city in cities:
            if pd.isna(city) or city == '':
                translations[city] = city
        
    except Exception as e:
        print(f"Translation error: {e}")
        translations = {city: city for city in cities}
    
    return translations

def translate_all_cities(unique_cities, batch_size=BATCH_SIZE, max_workers=MAX_WORKERS):
    """Translate all unique cities using Azure Translator with parallel processing"""
    print(f"Translating {len(unique_cities)} unique cities...")
    
    city_batches = [unique_cities[i:i + batch_size] 
                    for i in range(0, len(unique_cities), batch_size)]
    
    all_translations = {}
    completed = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_batch = {executor.submit(translate_batch_azure, batch): batch 
                          for batch in city_batches}
        
        for future in as_completed(future_to_batch):
            try:
                translations = future.result()
                all_translations.update(translations)
                completed += len(translations)
                progress = (completed * 100) // len(unique_cities)
                print(f"Progress: {completed}/{len(unique_cities)} cities ({progress}%)")
            except Exception as e:
                print(f"Batch processing error: {e}")
    
    return all_translations

def update_database_small_batches(translations, table_name, batch_size=DB_UPDATE_BATCH_SIZE):
    """
    Update database in SMALL batches to avoid query size limits
    Uses individual UPDATE statements, not CASE
    """
    if not translations:
        return
    
    valid_translations = {k: v for k, v in translations.items() 
                         if k and v and pd.notna(k) and pd.notna(v)}
    
    if not valid_translations:
        print("No valid translations to update")
        return
    
    total = len(valid_translations)
    print(f"Updating database with {total} translations in batches of {batch_size}...")
    
    # Convert to list for batching
    items = list(valid_translations.items())
    updated_count = 0
    conn = None
    
    try:
        for i in range(0, len(items), batch_size):
            batch = items[i:i + batch_size]
            
            # Create new connection for each batch
            if conn:
                try:
                    conn.close()
                except:
                    pass
            
            conn = get_connection()
            cursor = conn.cursor()
            
            # Update each item individually within transaction
            for original, translated in batch:
                try:
                    update_query = """
                    UPDATE {} 
                    SET geolocation_city_en = %s 
                    WHERE geolocation_city = %s 
                    AND (geolocation_city_en IS NULL OR geolocation_city_en = '')
                    """.format(table_name)
                    
                    cursor.execute(update_query, (translated, original))
                    updated_count += 1
                    
                except Exception as e:
                    print(f"Error updating '{original}': {e}")
                    continue
            
            # Commit batch
            conn.commit()
            cursor.close()
            
            # Progress update
            progress = ((i + len(batch)) * 100) // total
            print(f"Database update progress: {updated_count}/{total} ({progress}%)")
    
    except Exception as e:
        print(f"Database update error: {e}")
    finally:
        if conn:
            try:
                conn.close()
            except:
                pass
    
    print(f"✓ Successfully updated {updated_count} records")

def main():
    try:
        start_time = time.time()
        
        print("=" * 70)
        print("Azure SQL Geolocation Translation Tool - Azure Translator API")
        print("=" * 70)
        
        # Check configuration
        if AZURE_TRANSLATOR_KEY == 'your_translator_key_here':
            print("❌ ERROR: Please update AZURE_TRANSLATOR_KEY")
            print("\nGet your key from:")
            print("Azure Portal → Translator Resource → Keys and Endpoint → Key 1")
            return
        
        # Display current IP for troubleshooting
        print("\n📍 Getting your current IP address...")
        try:
            import requests
            current_ip = requests.get('https://api.ipify.org', timeout=5).text
            print(f"✓ Your current IP: {current_ip}")
            print("  (Use this IP if you need to add a firewall rule)")
        except:
            print("  (Could not detect IP)")
        
        print("\n🔌 Connecting to Azure SQL Server...")
        print(f"   Server: {SERVER}")
        print(f"   Database: {DATABASE}")
        
        conn = get_connection()
        cursor = conn.cursor()
        print("✓ Connected successfully!")
        
        # Check/create translation column
        print("\n📋 Checking table structure...")
        check_column_query = f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE TABLE_NAME = '{TABLE_NAME}' 
            AND COLUMN_NAME = 'geolocation_city_en'
        )
        BEGIN
            ALTER TABLE {TABLE_NAME}
            ADD geolocation_city_en NVARCHAR(255)
        END
        """
        cursor.execute(check_column_query)
        conn.commit()
        print("✓ Table structure verified")
        
        # Read untranslated unique cities
        print(f"\n📖 Reading untranslated cities from {TABLE_NAME}...")
        query = f"""
        SELECT DISTINCT geolocation_city 
        FROM {TABLE_NAME} 
        WHERE (geolocation_city_en IS NULL OR geolocation_city_en = '')
        AND geolocation_city IS NOT NULL
        AND geolocation_city != ''
        """
        df = pd.read_sql(query, conn)
        
        cursor.close()
        conn.close()
        
        total_unique = len(df)
        print(f"✓ Found {total_unique:,} unique cities to translate")
        
        if total_unique == 0:
            print("\n✅ All cities are already translated!")
            return
        
        # Get list of unique cities
        unique_cities = df['geolocation_city'].tolist()
        
        # Estimate characters
        total_chars = sum(len(str(city)) for city in unique_cities)
        print(f"\n📊 Estimated characters: {total_chars:,}")
        print(f"   Azure Free Tier: 2,000,000/month")
        print(f"   Percentage: {(total_chars/2000000)*100:.2f}%")
        
        if total_chars > 2000000:
            print("   ⚠️  WARNING: May exceed free tier limit")
        
        # Translate all unique cities
        print(f"\n🌐 Starting translation...")
        print(f"   Batch size: {BATCH_SIZE} cities/request")
        print(f"   Parallel workers: {MAX_WORKERS}")
        
        translations = translate_all_cities(unique_cities, BATCH_SIZE, MAX_WORKERS)
        
        # Update database in small batches
        print(f"\n💾 Updating database...")
        print(f"   Update batch size: {DB_UPDATE_BATCH_SIZE} cities")
        
        update_database_small_batches(translations, TABLE_NAME, DB_UPDATE_BATCH_SIZE)
        
        # Show sample results
        print("\n📋 Sample translations:")
        conn = get_connection()
        sample_query = f"""
        SELECT TOP 10 geolocation_city, geolocation_city_en 
        FROM {TABLE_NAME} 
        WHERE geolocation_city_en IS NOT NULL
        ORDER BY geolocation_city
        """
        sample_df = pd.read_sql(sample_query, conn)
        print(sample_df.to_string(index=False))
        conn.close()
        
        # Show statistics
        elapsed_time = time.time() - start_time
        print(f"\n" + "=" * 70)
        print(f"✅ TRANSLATION COMPLETED SUCCESSFULLY!")
        print("=" * 70)
        print(f"📊 Statistics:")
        print(f"   Unique cities translated: {total_unique:,}")
        print(f"   Characters processed: {total_chars:,}")
        print(f"   Time taken: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")
        if elapsed_time > 0:
            print(f"   Average speed: {total_unique/elapsed_time:.2f} cities/second")
        print("=" * 70)
        
    except pymssql.OperationalError as e:
        error_code = e.args[0] if e.args else None
        if error_code == 40615:
            print("\n❌ FIREWALL ERROR - Cannot connect to database")
            print("\nPlease follow the instructions above to fix the firewall.")
        else:
            print(f"\n❌ Database Error: {e}")
            import traceback
            traceback.print_exc()
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'pymssql'

SyntaxError: invalid syntax (914646726.py, line 1)

In [ ]:
import pandas as pd
import pymssql
import requests
import uuid
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Azure SQL Server Configuration
SERVER = 'almagraby.database.windows.net'
DATABASE = 'Olist_dataset'
USERNAME = 'DEPI_Projecte'
PASSWORD = 'Almaghraby@240'
TABLE_NAME = 'olist_geolocation_dataset_olist_geolocation_dataset'  # Table containing geolocation_city column

# Azure Translator Configuration
AZURE_TRANSLATOR_KEY = 'AunHlUi5xLkxOBfcQnwIYmiLxh2PztKTD5b5Kbfu4uiLHt1alQKdJQQJ99BJACF24PCXJ3w3AAAbACOGZEWL'  # From Azure Portal
AZURE_TRANSLATOR_REGION = 'uaenorth'  # Your translator region (e.g., eastus, westeurope)
AZURE_TRANSLATOR_ENDPOINT = 'https://almagraby.cognitiveservices.azure.com/'

# Performance Settings
BATCH_SIZE = 100
MAX_WORKERS = 5
DB_UPDATE_BATCH_SIZE = 50  # CRITICAL: Smaller batches for UPDATE statements

def get_connection():
    """Create a new database connection with retry logic"""
    max_retries = 3
    retry_delay = 5
    
    for attempt in range(max_retries):
        try:
            conn = pymssql.connect(
                server=SERVER,
                user=USERNAME,
                password=PASSWORD,
                database=DATABASE,
                port=1433,
                tds_version='7.4',
                timeout=60,
                login_timeout=60
            )
            return conn
        except pymssql.OperationalError as e:
            error_code = e.args[0] if e.args else None
            
            if error_code == 40615:
                # Firewall blocking
                print(f"\n⚠️  FIREWALL ERROR (Attempt {attempt + 1}/{max_retries})")
                print("=" * 70)
                print("Your Azure SQL Server firewall is blocking this connection.")
                print("\nTO FIX THIS:")
                print("1. Get your current IP:")
                
                try:
                    import requests
                    current_ip = requests.get('https://api.ipify.org', timeout=5).text
                    print(f"   Your IP: {current_ip}")
                except:
                    print("   Run: import requests; print(requests.get('https://api.ipify.org').text)")
                
                print("\n2. Add to Azure Firewall:")
                print("   • Azure Portal → SQL Server → Networking")
                print("   • Click '+ Add firewall rule'")
                print(f"   • Start IP: {current_ip if 'current_ip' in locals() else '<your_ip>'}")
                print(f"   • End IP: {current_ip if 'current_ip' in locals() else '<your_ip>'}")
                print("   • Click 'Save' and wait 2-5 minutes")
                print("\n3. OR Enable 'Allow Azure services' (Recommended):")
                print("   • Azure Portal → SQL Server → Networking")
                print("   • Toggle ON: 'Allow Azure services and resources to access this server'")
                print("   • Click 'Save'")
                print("=" * 70)
                
                if attempt < max_retries - 1:
                    print(f"\nWaiting {retry_delay} seconds before retry...")
                    time.sleep(retry_delay)
                else:
                    print("\n❌ Max retries reached. Please fix firewall and re-run.")
                    raise
            else:
                # Other connection errors
                print(f"\n❌ Connection Error: {e}")
                if attempt < max_retries - 1:
                    print(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    raise
        except Exception as e:
            print(f"\n❌ Unexpected Error: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                raise
    
    raise Exception("Failed to connect after all retries")

def translate_batch_azure(cities):
    """Translate a batch of cities using Azure Translator API"""
    if not cities:
        return {}
    
    valid_cities = [city for city in cities if pd.notna(city) and city != '']
    
    if not valid_cities:
        return {city: city for city in cities}
    
    path = '/translate'
    constructed_url = AZURE_TRANSLATOR_ENDPOINT + path
    
    params = {
        'api-version': '3.0',
        'from': 'auto',
        'to': 'en'
    }
    
    headers = {
        'Ocp-Apim-Subscription-Key': AZURE_TRANSLATOR_KEY,
        'Ocp-Apim-Subscription-Region': AZURE_TRANSLATOR_REGION,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    
    body = [{'text': str(city)} for city in valid_cities]
    translations = {}
    
    try:
        response = requests.post(constructed_url, params=params, headers=headers, json=body)
        response.raise_for_status()
        results = response.json()
        
        for city, result in zip(valid_cities, results):
            translated_text = result['translations'][0]['text']
            translations[city] = translated_text
        
        for city in cities:
            if pd.isna(city) or city == '':
                translations[city] = city
        
    except Exception as e:
        print(f"Translation error: {e}")
        translations = {city: city for city in cities}
    
    return translations

def translate_all_cities(unique_cities, batch_size=BATCH_SIZE, max_workers=MAX_WORKERS):
    """Translate all unique cities using Azure Translator with parallel processing"""
    print(f"Translating {len(unique_cities)} unique cities...")
    
    city_batches = [unique_cities[i:i + batch_size] 
                    for i in range(0, len(unique_cities), batch_size)]
    
    all_translations = {}
    completed = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_batch = {executor.submit(translate_batch_azure, batch): batch 
                          for batch in city_batches}
        
        for future in as_completed(future_to_batch):
            try:
                translations = future.result()
                all_translations.update(translations)
                completed += len(translations)
                progress = (completed * 100) // len(unique_cities)
                print(f"Progress: {completed}/{len(unique_cities)} cities ({progress}%)")
            except Exception as e:
                print(f"Batch processing error: {e}")
    
    return all_translations

def update_database_small_batches(translations, table_name, batch_size=DB_UPDATE_BATCH_SIZE):
    """
    Update database in SMALL batches to avoid query size limits
    Uses individual UPDATE statements, not CASE
    """
    if not translations:
        return
    
    valid_translations = {k: v for k, v in translations.items() 
                         if k and v and pd.notna(k) and pd.notna(v)}
    
    if not valid_translations:
        print("No valid translations to update")
        return
    
    total = len(valid_translations)
    print(f"Updating database with {total} translations in batches of {batch_size}...")
    
    # Convert to list for batching
    items = list(valid_translations.items())
    updated_count = 0
    conn = None
    
    try:
        for i in range(0, len(items), batch_size):
            batch = items[i:i + batch_size]
            
            # Create new connection for each batch
            if conn:
                try:
                    conn.close()
                except:
                    pass
            
            conn = get_connection()
            cursor = conn.cursor()
            
            # Update each item individually within transaction
            for original, translated in batch:
                try:
                    update_query = """
                    UPDATE {} 
                    SET geolocation_city_en = %s 
                    WHERE geolocation_city = %s 
                    AND (geolocation_city_en IS NULL OR geolocation_city_en = '')
                    """.format(table_name)
                    
                    cursor.execute(update_query, (translated, original))
                    updated_count += 1
                    
                except Exception as e:
                    print(f"Error updating '{original}': {e}")
                    continue
            
            # Commit batch
            conn.commit()
            cursor.close()
            
            # Progress update
            progress = ((i + len(batch)) * 100) // total
            print(f"Database update progress: {updated_count}/{total} ({progress}%)")
    
    except Exception as e:
        print(f"Database update error: {e}")
    finally:
        if conn:
            try:
                conn.close()
            except:
                pass
    
    print(f"✓ Successfully updated {updated_count} records")

def main():
    try:
        start_time = time.time()
        
        print("=" * 70)
        print("Azure SQL Geolocation Translation Tool - Azure Translator API")
        print("=" * 70)
        
        # Check configuration
        if AZURE_TRANSLATOR_KEY == 'your_translator_key_here':
            print("❌ ERROR: Please update AZURE_TRANSLATOR_KEY")
            print("\nGet your key from:")
            print("Azure Portal → Translator Resource → Keys and Endpoint → Key 1")
            return
        
        # Display current IP for troubleshooting
        print("\n📍 Getting your current IP address...")
        try:
            import requests
            current_ip = requests.get('https://api.ipify.org', timeout=5).text
            print(f"✓ Your current IP: {current_ip}")
            print("  (Use this IP if you need to add a firewall rule)")
        except:
            print("  (Could not detect IP)")
        
        print("\n🔌 Connecting to Azure SQL Server...")
        print(f"   Server: {SERVER}")
        print(f"   Database: {DATABASE}")
        
        conn = get_connection()
        cursor = conn.cursor()
        print("✓ Connected successfully!")
        
        # Check/create translation column
        print("\n📋 Checking table structure...")
        check_column_query = f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE TABLE_NAME = '{TABLE_NAME}' 
            AND COLUMN_NAME = 'geolocation_city_en'
        )
        BEGIN
            ALTER TABLE {TABLE_NAME}
            ADD geolocation_city_en NVARCHAR(255)
        END
        """
        cursor.execute(check_column_query)
        conn.commit()
        print("✓ Table structure verified")
        
        # Read untranslated unique cities
        print(f"\n📖 Reading untranslated cities from {TABLE_NAME}...")
        query = f"""
        SELECT DISTINCT geolocation_city 
        FROM {TABLE_NAME} 
        WHERE (geolocation_city_en IS NULL OR geolocation_city_en = '')
        AND geolocation_city IS NOT NULL
        AND geolocation_city != ''
        """
        df = pd.read_sql(query, conn)
        
        cursor.close()
        conn.close()
        
        total_unique = len(df)
        print(f"✓ Found {total_unique:,} unique cities to translate")
        
        if total_unique == 0:
            print("\n✅ All cities are already translated!")
            return
        
        # Get list of unique cities
        unique_cities = df['geolocation_city'].tolist()
        
        # Estimate characters
        total_chars = sum(len(str(city)) for city in unique_cities)
        print(f"\n📊 Estimated characters: {total_chars:,}")
        print(f"   Azure Free Tier: 2,000,000/month")
        print(f"   Percentage: {(total_chars/2000000)*100:.2f}%")
        
        if total_chars > 2000000:
            print("   ⚠️  WARNING: May exceed free tier limit")
        
        # Translate all unique cities
        print(f"\n🌐 Starting translation...")
        print(f"   Batch size: {BATCH_SIZE} cities/request")
        print(f"   Parallel workers: {MAX_WORKERS}")
        
        translations = translate_all_cities(unique_cities, BATCH_SIZE, MAX_WORKERS)
        
        # Update database in small batches
        print(f"\n💾 Updating database...")
        print(f"   Update batch size: {DB_UPDATE_BATCH_SIZE} cities")
        
        update_database_small_batches(translations, TABLE_NAME, DB_UPDATE_BATCH_SIZE)
        
        # Show sample results
        print("\n📋 Sample translations:")
        conn = get_connection()
        sample_query = f"""
        SELECT TOP 10 geolocation_city, geolocation_city_en 
        FROM {TABLE_NAME} 
        WHERE geolocation_city_en IS NOT NULL
        ORDER BY geolocation_city
        """
        sample_df = pd.read_sql(sample_query, conn)
        print(sample_df.to_string(index=False))
        conn.close()
        
        # Show statistics
        elapsed_time = time.time() - start_time
        print(f"\n" + "=" * 70)
        print(f"✅ TRANSLATION COMPLETED SUCCESSFULLY!")
        print("=" * 70)
        print(f"📊 Statistics:")
        print(f"   Unique cities translated: {total_unique:,}")
        print(f"   Characters processed: {total_chars:,}")
        print(f"   Time taken: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")
        if elapsed_time > 0:
            print(f"   Average speed: {total_unique/elapsed_time:.2f} cities/second")
        print("=" * 70)
        
    except pymssql.OperationalError as e:
        error_code = e.args[0] if e.args else None
        if error_code == 40615:
            print("\n❌ FIREWALL ERROR - Cannot connect to database")
            print("\nPlease follow the instructions above to fix the firewall.")
        else:
            print(f"\n❌ Database Error: {e}")
            import traceback
            traceback.print_exc()
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()